FLAME - Fuzzy clustering by Local Approximation of MEmbership

In [1]:
from __future__ import print_function

import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.utils import check_array
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import normalize
from math import sqrt
import datetime

In [2]:
class FLAME(BaseEstimator, ClusterMixin):
    def __init__(self, metric="euclidean", cluster_neighbors=5, iteration_neighbors=5, max_iter=np.inf, eps=1e-10, thd=-2, verbose=0):
        self.metric = metric
        self.cluster_neighbors = cluster_neighbors
        self.iteration_neighbors = iteration_neighbors
        self.max_iter = max_iter
        self.eps = eps
        self.thd = thd
        self.verbose = verbose
    def _get_nearest(self, distances, n_neighbors, n_samples):
# Make a numpy arange for iteration purposes.
        sample_range = np.arange(n_samples)[:, None]
# Do an introsort on each row of the distances matrix to put the nth smallest distance in the nth position and all
# smaller elements before it. Then keep only the first n+1 elements (including the element itself which will have
# distance 0 from itself and is removed later).
        nearest_np = np.argpartition(distances, n_neighbors, axis=1)
        nearest_np = nearest_np[:, :n_neighbors + 1]
# Find the largest distance of the kth closest points.
        largest_distance = distances[sample_range, nearest_np[sample_range, -1]]
# Make two arrays of sets the first containing only the n nearest other elements to each element not
# including the element itself and the second containing the same plus any other elements tied for nth nearest
# again excluding the element itself (though if there are k other elements all 0 distance away other problems
# will result).
        nearest = []
        nearest_with_ties = []
        for i in range(n_samples):
            ties_for_largest_distance = np.where(distances[i] == largest_distance[i])
            nearest.append(set(nearest_np[i, :].tolist()))
            nearest[-1].remove(i)
            ties_for_largest_distance = set(ties_for_largest_distance[0].tolist())
            ties_for_largest_distance.discard(i)
            nearest_with_ties.append(nearest[i] | ties_for_largest_distance)
        return nearest, nearest_with_ties
    def _get_densities(self, distances, nearest, n_samples):
# Make a numpy arange for iteration purposes.
        sample_range = np.arange(n_samples)[:, None]
        nearest_np = np.array([list(s) for s in nearest])
        n_shortest_distances = distances[sample_range, nearest_np]
        local_distance_sums = n_shortest_distances.sum(axis=1)
        largest_local_sum = local_distance_sums.max(axis=0)
        densities = np.asarray(largest_local_sum / local_distance_sums)
        return densities
    def _get_supports(self, densities, nearest_with_ties, n_samples):
        density_sum = densities.sum()
        density_mean = density_sum / n_samples
        density_sum2 = (densities * densities).sum()
        thd = density_mean + self.thd * sqrt(density_sum2 / n_samples - density_mean * density_mean)
        csos = []
        outliers = []
        remaining = []
        for i in range(n_samples):
            if densities[i] < thd:
                outliers.append(i)
            elif densities[i] > densities[list(nearest_with_ties[i])].max():
                csos.append(i)
            else:
                remaining.append(i)
        return csos, outliers, remaining
    def _get_weights(self, distances, nearest_with_ties, fixed, n_samples):
        nearest_with_ties = [sorted(list(s)) for s in nearest_with_ties]
        weights = lil_matrix((n_samples, n_samples))
        for i in range(n_samples):
            if i in fixed:
                weights[i, i] = 1
            else:
                for j in nearest_with_ties[i]:
                    weights[i, j] = distances[i, j]
            if self.verbose: print("Assigned weights {0}.".format(i))
        weights = weights.tocsr()
        weights = normalize(weights, norm='l1', axis=1, copy=False)
        return weights
    def _get_starting_membership(self, csos, outliers, fixed, n_samples):
        M = len(csos) + 1
        starting_membership = np.zeros(shape=(n_samples, M))
        general_row = np.ndarray(shape=(1, M))
        general_row.fill(1. / M)
        for i in range(n_samples):
             if i not in fixed:
                starting_membership[i, :] = general_row
        for index, value in enumerate(csos):
            starting_membership[value, index] = 1
        for i in outliers:
            starting_membership[i, -1] = 1
        return starting_membership
    def _flame(self, X):

#Pass Numpy or Pandas array of data as X. As metric pass any string as in sklearn.metrics.pairwise.pairwise_distances
#or a callable on pairs of members of X. FLAME is computed with n_neighbors until max_iter or convergence up to eps.
#thd is the threshold for outliers: Any element which has less than mean(density) + thd * std(density) will be an outlier.

        if sparse.issparse(X) and self.metric not in {"precomputed", "cityblock", "cosine", "euclidean", "l1", "l2",
                                                     "manhattan"} and not callable(self.metric): 
                    raise TypeError("The metric {0} does not support sparse data.".format(self.metric))
# Convert pandas objects to numpy arrays.
        if 'pandas' in str(X.__class__):
                X = X.values
        X = check_array(X, accept_sparse="csr", dtype=None)
# Get the number of samples. We use this a lot.
        n_samples, _ = X.shape
        distances = pairwise_distances(X, metric=self.metric)
        nearest, nearest_with_ties = self._get_nearest(distances, self.cluster_neighbors, n_samples)
        if self.verbose: print("Got distances and nearest.")
        densities = self._get_densities(distances, nearest, n_samples)
        if self.verbose: print("Got densities.")
        csos, outliers, remaining = self._get_supports(densities, nearest_with_ties, n_samples)
        if self.verbose: print("Got suppports.")
        if self.verbose: print("There are {0} clusters and {1} outliers.".format(len(csos), len(outliers)))
        fixed = set(csos) | set(outliers)
        _, nearest_with_ties_for_iteration = self._get_nearest(distances, self.iteration_neighbors, n_samples)
        weights = self._get_weights(distances, nearest_with_ties_for_iteration, fixed, n_samples)
        if self.verbose: print("Got weights.")
        membership_proba = self._get_starting_membership(csos, outliers, fixed, n_samples)
        if self.verbose: print("Got starting memberships.")
        i = 0
        while i < self.max_iter:
            lastMembership = membership_proba.copy()
            membership_proba = weights.dot(membership_proba)
            delta = np.absolute(membership_proba - lastMembership).max()
            i += 1
            if self.verbose: print("Done iteration {0}.".format(i))
            if delta < self.eps:
                break
        num_clusters = membership_proba.shape[1] - 1
 # Get cluster assignment.
        pred = np.argmax(membership_proba, axis=1)
 # Replace predictions of the outlier group with -1.
        pred[pred == num_clusters] = -1
        return membership_proba, pred, csos, outliers, densities
    def fit(self, X):
        self.membership_proba_, self.labels_, self.csos_, self.outliers_, self.densities_ = self._flame(X)
#改改改！！                 #self._flame(X)
        return self
            
    def fit_predict(self, X, y=None):
            y = self.fit(X).labels_
            return y
            
    def fit_predict_proba(self, X, y=None):
            y = self.fit(X).membership_proba_
            return y


In [88]:
import pandas as pd     #引入pandas包
data=pd.read_table('test10000x200.txt',sep='\t',header=0,index_col=0)  #读入txt文件，分隔符为\t

,TCGA-2F-A9KO-01,TCGA-2F-A9KQ-01,TCGA-2F-A9KR-01,TCGA-2F-A9KT-01,TCGA-4Z-AA7M-01,TCGA-4Z-AA7O-01,TCGA-4Z-AA7Q-01,TCGA-4Z-AA7R-01,TCGA-4Z-AA7S-01,TCGA-4Z-AA7W-01,...,TCGA-G2-A2EL-01,TCGA-G2-A2EO-01,TCGA-G2-A2ES-01,TCGA-G2-A3IB-01,TCGA-G2-A3IE-01,TCGA-G2-A3VY-01,TCGA-G2-AA3B-01,TCGA-GC-A3BM-01,TCGA-GC-A3I6-01,TCGA-GC-A3WC-01
TCGA/BLCA/mRNA,,,,,,,,,,,,,,,,,,,,,
ARHGEF10L,0.221008,1.173308,1.090208,0.955008,1.116208,0.814508,0.274908,0.300708,1.280308,-0.223792,...,-0.668592,-0.045492,-1.447892,0.045408,1.750408,1.151608,1.647908,1.261608,-0.565092,-0.499692
HIF3A,4.428374,-5.554726,0.971574,-1.481926,-1.632426,-3.235726,0.010774,-1.583326,-3.837526,-1.013426,...,-4.866626,-2.260626,-5.554726,-3.415926,-3.524326,2.065274,-3.715226,-4.400026,-4.431926,0.797874
RNF17,-0.531035,-0.531035,-0.531035,0.115365,-0.531035,-0.531035,0.202165,1.344865,-0.531035,-0.531035,...,-0.146335,0.043665,-0.531035,0.281565,-0.531035,3.285665,-0.531035,-0.531035,3.530965,3.864765
RNF10,0.720328,1.647728,0.256228,-0.039972,0.002728,0.473128,0.478528,0.399028,0.320628,0.654528,...,0.501528,0.404828,0.039328,0.353328,0.342328,0.033628,0.055128,0.399028,0.183528,-0.303272
RNF11,-0.408078,0.518622,0.586622,0.746322,-0.127278,0.247522,0.049322,-0.675778,0.560522,-1.071078,...,0.588822,-0.234678,-0.157678,0.718822,0.031022,0.012122,-1.066078,0.297522,-0.549378,-0.666778


In [89]:
#用FLAME聚类：记录时间
starttime = datetime.datetime.now()
clusterer = FLAME(metric="braycurtis",cluster_neighbors=10, iteration_neighbors=10, verbose=0, thd=-3)
clusterer = clusterer.fit(data)
pred = clusterer.labels_
df = pd.DataFrame(data)
endtime = datetime.datetime.now()
print(endtime - starttime)
print(len(set(pred)))

In [90]:
#聚类结果整体写入csv
df["V1"] = pred
df = df.sort_values(by="V1" , ascending=True)#升序0.1.2.3
df1=df.V1
df1.to_csv("result/test.csv",index=True,sep=',')

In [91]:
#按照聚类结果分别写入txt文件
column_uniques = df['V1'].unique()
for column in column_uniques:
    temp_data = df[df['V1'].isin([column])] #按照列的某个值取一个dataframe 
    file_name = 'module'+str(column+1)+'.txt'
    file_path ='result/'+file_name
    temp_data.to_csv(file_path, sep='\t', index=True)
